<a href="https://colab.research.google.com/github/soyRex-codes/New_Bommer_Bot-450/blob/main/New_Bot_Bommer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Summary of 5 Steps in this codebase:
Step 1: Setup and Installation: This cell installs all necessary libraries at once.

Step 2: Load and Process Your Document
This cell will handle uploading your PDF, extracting the text, and storing it. This is a crucial step to ensure the local_data_text is always available for the next steps.

Step 3: Set Up the Vector Database (The "Retriever")
This cell chunks the extracted text and stores it as embeddings in a ChromaDB vector database.

Step 4: Implement the Full RAG Pipeline
This cell contains the final logic. It defines functions to retrieve context and generate a final answer, then sets up the language model pipeline.

Step 5: Create an Interactive Chat Interface
This final cell provides a simple loop to chat with your document-aware model.

#JSON file example for syntax help

[
  {
    "id": "cs_req_001",
    "question_variants": [
      "What are the degree requirements for computer science?",
      "How do I graduate with a CS degree?",
      "What are the main requirements for the computer science program?",
      "Tell me about the graduation criteria for CS."
    ],

    "context_answer": "To graduate with a Computer Science degree from Missouri State University, a student must complete at least 120 hours of courses. A minimum of 40 of these hours must be in courses numbered 300 and above. Additionally, a student must achieve a score of at least the 50th percentile on the Computer Science Major Field Test (MFT)."
  },

  {
    "id": "mascot_info_001",
    "question_variants": [
      "Who is the university mascot?",
      "What is the mascot's name?",
      "Tell me about Boomer the Bear."
    ],
    
    "context_answer": "The official mascot of Missouri State University is Boomer the Bear. He is a beloved figure known for his energetic and friendly personality at sporting events and other university functions."
  }
]

In [ ]:
#Step 1 Install all required packages: This cell installs all necessary libraries at once.
!pip install transformers torch accelerate
!pip install sentence-transformers chromadb
#!pip install pdfminer.six #no need, since we switched to ,json processing than pdf processing #outdate

print("All necessary packages for the JSON-based RAG pipeline are installed.")

All necessary packages for the JSON-based RAG pipeline are installed.


In [ ]:
# Step 2 & 3: Load JSON and Build the Vector DB

'''This single block of code will now handle loading the JSON and populating the vector database. It replaces the old PDF processing and database setup cells.'''
import json
import chromadb
from sentence_transformers import SentenceTransformer
import uuid
from google.colab import files

def load_and_build_from_json_qa(collection_name="json_qa_collection"):
    """
    Handles uploading the JSON Q&A file, processing it, and building the vector database.

    Args:
        collection_name (str): The name for the ChromaDB collection.

    Returns:
        chromadb.Collection: The ChromaDB collection object, or None on failure.
    """
    print("Please upload your `knowledge_base.json` file.")
    uploaded = files.upload()

    if not uploaded:
        print("No file uploaded.")
        return None

    file_name = next(iter(uploaded))

    if not file_name.lower().endswith('.json'):
        print("Error: The uploaded file is not a .json file.")
        return None

    print("Processing JSON file and building vector database...")

    # 1. Read and parse the JSON file
    try:
        with open(file_name, 'r') as f:
            qa_data = json.load(f)
    except Exception as e:
        print(f"Error reading or parsing JSON file: {e}")
        return None

    # 2. Create formatted text blocks for embedding
    # We combine a primary question and the answer to create a rich context for the embedding.
    documents_to_embed = []
    for item in qa_data:
        # We use the first question variant as the "primary" question for the text block
        primary_question = item["question_variants"][0]
        answer = item["context_answer"]
        formatted_doc = f"Question: {primary_question}\\nAnswer: {answer}"
        documents_to_embed.append(formatted_doc)

    if not documents_to_embed:
        print("No documents were created from the JSON file. Please check its structure.")
        return None

    print(f"Created {len(documents_to_embed)} text blocks to be embedded.")

    # 3. Load embedding model and generate embeddings
    embedding_model = SentenceTransformer('all-MiniLM-L6-v2')
    doc_embeddings = embedding_model.encode(documents_to_embed, show_progress_bar=True)

    # 4. Initialize ChromaDB and create collection
    client = chromadb.Client()
    if collection_name in [c.name for c in client.list_collections()]:
        client.delete_collection(name=collection_name)

    collection = client.create_collection(name=collection_name)

    # 5. Add documents to the collection
    ids = [item['id'] for item in qa_data] # Use the IDs from our JSON file
    collection.add(
        embeddings=doc_embeddings.tolist(),
        documents=documents_to_embed, # We store the formatted block
        ids=ids
    )

    print(f"Successfully added {collection.count()} documents to the '{collection_name}' collection.")
    return collection

# --- Execute the new function ---
collection = load_and_build_from_json_qa()

Please upload your `knowledge_base.json` file.


Saving Connect_to_MSU_WIFI_Network.json to Connect_to_MSU_WIFI_Network.json
Processing JSON file and building vector database...
Created 1 text blocks to be embedded.


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Successfully added 1 documents to the 'json_qa_collection' collection.


In [ ]:
#Step 4: Implement the Full RAG Pipeline
'''This cell contains the final logic. It defines functions to retrieve context and generate a final answer, then sets up the language model pipeline.'''
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from sentence_transformers import SentenceTransformer

# --- 1. Load All Models at the Start ---

# Load the main LLM and tokenizer for generation
print("Loading the main language model (TinyLlama)...")
model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16,
    device_map="auto"
)

# Create the text-generation pipeline
rag_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer
)

# Editing Needed: Load the embedding model for the retriever ONCE
print("Loading the embedding model for retrieval (all-MiniLM-L6-v2)...")
# Check for CUDA availability and set the device accordingly
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")
embedding_model_retriever = SentenceTransformer('all-MiniLM-L6-v2', device=device)


# --- 2. Define the Core RAG Functions ---

def retrieve_context(query, db_collection, embedder, n_results=5):
    """
    Retrieves the most relevant document chunks from the vector database.

    Args:
        embedder: The loaded SentenceTransformer model instance.
    """
    query_embedding = embedder.encode(query).tolist()
    results = db_collection.query(
        query_embeddings=[query_embedding],
        n_results=n_results
    )
    return "\n---\n".join(results['documents'][0]) # Use a separator for clarity

def generate_rag_response(query, db_collection):
    """
    Generates a response by retrieving context and feeding it to the LLM.
    """
    if not db_collection:
        return "The vector database is not set up. Please run the data loading step first."

    # 1. Retrieve relevant context using the pre-loaded embedding model
    context = retrieve_context(query, db_collection, embedding_model_retriever)

    # 2. Create the strict, refined prompt
    messages = [
        {
            "role": "system",
            "content": "You are a friendly helpful assistant. Answer the user's question specifically based ONLY on the provided context, only use other info if that's going to make the answer sound much better, make the answer sounds more human anc clear, but don't compromise the factual quality of the provided context... Do not use any of your outside knowledge."},
        {
            "role": "user",
            "content": f"Context:\n{context}\n\nQuestion: {query}"
        },
    ]
    prompt = rag_pipeline.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

    # 3. Generate the response
    outputs = rag_pipeline(
        prompt,
        max_new_tokens=256,
        do_sample=True,
        temperature=0.2, # lower temperature for less creative and more factual answers
        top_p=0.95
    )

    # 4. Clean up the output
    full_response = outputs[0]['generated_text']
    # The split string should exactly match the model's chat template output
    if "<|assistant|>" in full_response:
        assistant_response = full_response.split("<|assistant|>")[1].strip()
    else:
        # Fallback for unexpected template changes
        assistant_response = "Could not parse the model's response."

    return assistant_response

# --- Example Test ---
# Editing Needed: Use a more relevant test query based on our JSON data.
if 'collection' in locals() and collection is not None:
    print("\n--- Running Example Test ---")
    test_query = "How to connect to msu wifi?"
    print(f"Testing with query: '{test_query}'")
    response = generate_rag_response(test_query, collection)
    print(f"Model Response: {response}")
else:
    print("\nSkipping example test because the 'collection' variable was not found.")

Loading the main language model (TinyLlama)...


Device set to use cuda:0


Loading the embedding model for retrieval (all-MiniLM-L6-v2)...
Using device: cuda

--- Running Example Test ---
Testing with query: 'How to connect to msu wifi?'
Model Response: To connect to the MSU Open Wi-Fi network on Android devices, follow these steps:

1. Go to "Settings" on your Android device.
2. Select "Wi-Fi" from the list of options.
3. When prompted for information, set the EAP method to "PEAP" (Pre-Shared Key), the authentication method to "MSCHAPv2" (Microsoft Challenge-Handshake Authentication Protocol version 2), the CA certificate to "Do Not Validate" (not required), the user certificate to "Unspecified" (not required), and your Missouri State account ID (abc123) or if you are at the West Plains Campus (use MissouriStateAccountID@wp) in the User Certificate field.
4. If prompted to enter a domain name, enter missouristate.edu in the field.
5. Enter your password in the Password field.
6. If you are prompted to enter a domain name, enter missouristate.edu in the field

In [ ]:
#Step 5: Create an Interactive Chat Interface
''' --- This final cell provides a robust loop to chat with your document-aware model. --- '''

# Check1: checks that the RAG pipeline is actually ready before starting the chat.
if 'collection' not in locals() or collection is None:
    print("ERROR: The 'collection' object was not found.")
    print("Please ensure you have successfully run the previous steps to upload a document and build the vector database.")
else:
    print("--- Document RAG Chat Interface ---")
    print("The RAG system is ready. Enter your question, or type 'quit' to exit.")
    print("="*40)

    while True:
        user_input = input("You: ")

        # check 2: Handle the 'quit' command and check for empty input.
        if user_input.lower() == 'quit':
            break
        if not user_input.strip(): # Catches empty strings or just spaces
            continue

        # check 3: Use a try/except block for robust error handling.
        try:
            print("Assistant is thinking...") # Provides a better user experience
            response = generate_rag_response(user_input, collection)
            print(f"Assistant: {response}\n")
        except Exception as e:
            print(f"An unexpected error occurred: {e}")
            print("Please try a different question.\n")

    print("Session ended.")

--- Document RAG Chat Interface ---
The RAG system is ready. Enter your question, or type 'quit' to exit.
You: how do i connect to wifi
Assistant is thinking...
Assistant: To connect to the MSU Open Wi-Fi network on Android, follow these steps:

1. Go to "Wi-Fi" in the settings menu on your Android device.
2. When prompted for information, set the EAP method to "PEAP" and the authentication method to "MSCHAPv2".
3. Choose "Do Not Validate" for the CA certificate.
4. Enter your Missouri State account ID (abc123) or if you are at the West Plains Campus, your Missouri State account ID (abc123) or use the MissouriStateAccountID@wp as your username.
5. Enter your password in the password field.
6. If you are prompted to enter a domain name, enter missouristate.edu in the field.
7. Click "Connect" to connect to the Wi-Fi network.

Note that not all versions of Android OS require this step. If you are prompted to enter a domain name, enter missouristate.edu in the field.



KeyboardInterrupt: Interrupted by user